# This is an earlier branch of code where I separated out embarked into 3 numeric columns with 1 or 0 instead of 1 column with ports 1-3. 

performance seems better with a single column, so I'll continue with that model.  

In [240]:
from __future__ import print_function

#data handling/prediction
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.externals.six import StringIO
import os

#visualization
%matplotlib inline
import seaborn as sns
from IPython.display import Image
import pydotplus
from sklearn.tree import export_graphviz

#set variable to actually use graphviz
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

#import data
trainData  = pd.read_csv("../data/train.csv")
#testData = pd.read_csv("../data/test.csv")
#can't use the provided test data at this point 
#because don't have survival values to calculate RMSE

In [241]:
trainData.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [242]:
colList = trainData.columns.tolist()
colList

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [243]:
colList = ['Name','Sex','Ticket','Cabin','Embarked','PassengerId','Survived','Age','Pclass','SibSp','Parch','Fare']

In [244]:
working = trainData[colList]
working.head()

,Name,Sex,Ticket,Cabin,Embarked,PassengerId,Survived,Age,Pclass,SibSp,Parch,Fare
0,"Braund, Mr. Owen Harris",male,A/5 21171,NaN,S,1,0,22.0,3,1,0,7.2500
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C,2,1,38.0,1,1,0,71.2833
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,NaN,S,3,1,26.0,3,0,0,7.9250
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S,4,1,35.0,1,1,0,53.1000
4,"Allen, Mr. William Henry",male,373450,NaN,S,5,0,35.0,3,0,0,8.0500


In [245]:
working.shape

(891, 12)

In [246]:
trainData.shape

(891, 12)

In [247]:
#change gender to number
working["genM"]=(working.Sex=='male').astype(int)
working.genM.value_counts()

C:\Users\Elizabeth\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


1    577
0    314
Name: genM, dtype: int64

In [248]:
#confirm casting was done right
working.Sex.value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [249]:
working.Embarked.unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [250]:
#change Embarked to number
working["PortS"]=(working.Embarked=='S').astype(int)
working.PortS.value_counts()

C:\Users\Elizabeth\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


1    644
0    247
Name: PortS, dtype: int64

In [251]:
working["PortC"]=(working.Embarked=='C').astype(int)
working.PortC.value_counts()

C:\Users\Elizabeth\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


0    723
1    168
Name: PortC, dtype: int64

In [252]:
working["PortQ"]=(working.Embarked=='Q').astype(int)
working.PortQ.value_counts()

C:\Users\Elizabeth\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


0    814
1     77
Name: PortQ, dtype: int64

In [253]:
working.columns.tolist()

['Name',
 'Sex',
 'Ticket',
 'Cabin',
 'Embarked',
 'PassengerId',
 'Survived',
 'Age',
 'Pclass',
 'SibSp',
 'Parch',
 'Fare',
 'genM',
 'PortS',
 'PortC',
 'PortQ']

In [254]:
#exclude Age in col 7 for now - the nulls are throwing error when try to calculate RMSE
X, y = working.iloc[:, 8:], working.iloc[:, 6]

In [255]:
#for development purposes, use test_train_split on the cleaned train data so I have the survival value....
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=.4, random_state=1234)

In [256]:
#define features
feature_names_tr = X_train.columns.tolist()
feature_names_te = X_test.columns.tolist()

In [257]:
feature_names_tr

['Pclass', 'SibSp', 'Parch', 'Fare', 'genM', 'PortS', 'PortC', 'PortQ']

In [258]:
feature_names_te

['Pclass', 'SibSp', 'Parch', 'Fare', 'genM', 'PortS', 'PortC', 'PortQ']

#*NOTES TO SELF*#

11pm 19Feb18 - I'm not sure why it's throwing a 'convert string to float' error.  I think X_train somehow still has all the fields that I dropped from trainData.  I don't know how or why, though. try running with only Pclass, SibSp and Parch fields left.


Also still need to do analysis of the features and normalize/scale them (e.g. if keep fare, it and age are out of scale with the port and passenger class).  

In [259]:
#Random forest of 500 trees
rf = RandomForestRegressor(n_estimators=500, bootstrap=True, oob_score=True, random_state=123)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("Random Forest RMSE, unscaled & ignoring Age:", np.sqrt(mean_squared_error(y_test, y_pred_rf)))


Random Forest RMSE, unscaled & ignoring Age: 0.406238058293


# Now with scaling....

In [260]:
#let's try that with scaling, since Fare can be much larger than 1. 
sc = StandardScaler()
X_scaled = X.copy()
X_scaled[feature_names_tr] = sc.fit_transform(X[feature_names_tr])

In [261]:
X_scaled.head()

,Pclass,SibSp,Parch,Fare,genM,PortS,PortC,PortQ
0,0.827377,0.432793,-0.473674,-0.502445,0.737695,0.619306,-0.482043,-0.307562
1,-1.566107,0.432793,-0.473674,0.786845,-1.355574,-1.614710,2.074505,-0.307562
2,0.827377,-0.474545,-0.473674,-0.488854,-1.355574,0.619306,-0.482043,-0.307562
3,-1.566107,0.432793,-0.473674,0.420730,-1.355574,0.619306,-0.482043,-0.307562
4,0.827377,-0.474545,-0.473674,-0.486337,0.737695,0.619306,-0.482043,-0.307562


In [262]:
X_sc_train, X_sc_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=.4, random_state=1234)
rf = RandomForestRegressor(n_estimators=500, bootstrap=True, oob_score=True, random_state=123)
rf.fit(X_sc_train, y_train)
y_pred_rf = rf.predict(X_sc_test)

print("Random Forest RMSE, scaled, ignoring Age:", np.sqrt(mean_squared_error(y_test, y_pred_rf)))


Random Forest RMSE, scaled, ignoring Age: 0.406013645752


so scaling didn't make much difference (0.40601 with vs. 0.40623 without)
# dealing with missing ages

In [263]:
#how often is Age 'NaN'?
working_noAge = working[working.Age >= 0]
working_noAge.shape
#working.head()

(714, 16)

OK, so of the 891 samples, (819-714=) 105 of them don't have an age. 

# when have time....
fix by replacing nulls with mean based on gender

In [264]:
working.head()

,Name,Sex,Ticket,Cabin,Embarked,PassengerId,Survived,Age,Pclass,SibSp,Parch,Fare,genM,PortS,PortC,PortQ
0,"Braund, Mr. Owen Harris",male,A/5 21171,NaN,S,1,0,22.0,3,1,0,7.2500,1,1,0,0
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C,2,1,38.0,1,1,0,71.2833,0,0,1,0
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,NaN,S,3,1,26.0,3,0,0,7.9250,0,1,0,0
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S,4,1,35.0,1,1,0,53.1000,0,1,0,0
4,"Allen, Mr. William Henry",male,373450,NaN,S,5,0,35.0,3,0,0,8.0500,1,1,0,0
